In [1]:
import pandas as pd
import os
import json
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
doi_index = []
text = []

In [3]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

In [4]:
pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

In [7]:
labels = [0]*267

In [14]:
df = pd.DataFrame({'data': text, 'labels': labels})
print(df.loc[0:9])

                                                data  labels
0  10.1021/ic501413r Metal–organic frameworks (MO...       0
1  10.1021/jp102881e  Top of PageAbstractIntroduc...       0
2  10.1021/acs.analchem.5b00391 Metal–organic fra...       0
3  10.1021/ie071645b  Top of PageAbstract1. Intro...       0
4  10.1021/ic502478u Metal–organic frameworks (MO...       0
5  10.1021/cm4034319 Top of PageAbstractIntroduct...       0
6  10.1021/acs.langmuir.5b04185 In the original a...       0
7  10.1021/ie500310c Top of PageAbstractIntroduct...       0
8  10.1021/ie051056a  Top of PageAbstractIntroduc...       0
9  10.1021/ja5022014 Top of PageAbstractIntroduct...       0
